In [1328]:
import pandas as pd

In [1329]:
train=pd.read_csv('../train_df_errno.csv')
test=pd.read_csv('../test_df.csv')
sub=pd.read_csv('../sample_submission.csv')
age=pd.read_csv('../age_gender_info.csv')
train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

### 전용면적에 제한두기

In [1330]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

In [1331]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [1332]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

### 항목 묶어서 확인하기/ 이상치 제거해야할까?!

In [1333]:
#주차면수 대비 등록차량 비율이 높을 수록 주차수요가 높을것이다.
train['주차면수 대비 등록차량 비율']=train['등록차량수']/train['단지내주차면수']
test['주차면수 대비 등록차량 비율']=train['등록차량수']/test['단지내주차면수']

In [1334]:
#주차면수 대비 총세대수가 높을 수록 주차수요가 높을것이다
train['주차면수 대비 총세대수']=train['단지내주차면수']/train['총세대수']
test['주차면수 대비 총세대수']=test['단지내주차면수']/test['총세대수']

In [1335]:
#거주율이 높을 수록 주차수요가 높을 것이다.
train['거주율']=1-(train['공가수']/train['총세대수'])
test['거주율']=1-(test['공가수']/test['총세대수'])

### 10분내 버스와 지하철이 많은 주거지역은 주차수요가 낮을것이다

In [1336]:
null_sub=train.loc[train['10분내지하철수'].isnull()].sort_values(by=['지역'])
null_sub['지역'].unique()

array(['경상남도', '대전광역시', '충청남도'], dtype=object)

In [1337]:
null_sub_test=test.loc[test['10분내지하철수'].isnull()].sort_values(by=['지역'])
null_sub_test['지역'].unique()

array(['대전광역시', '충청남도'], dtype=object)

In [1338]:
daejeon=train.loc[train['지역']=='대전광역시'].sort_values(by=['10분내지하철수'])
daejeon['10분내지하철수'].value_counts()

1.0    125
0.0     48
Name: 10분내지하철수, dtype: int64

In [1339]:
daejeon_test=test.loc[train['지역']=='대전광역시'].sort_values(by=['10분내지하철수'])
daejeon_test['10분내지하철수'].value_counts()

0.0    153
2.0      2
Name: 10분내지하철수, dtype: int64

In [1340]:
daejeon_test=test.loc[train['지역']=='대전광역시'].sort_values(by=['10분내지하철수'])
daejeon_test['10분내지하철수'].value_counts()

0.0    153
2.0      2
Name: 10분내지하철수, dtype: int64

In [1341]:
train['10분내지하철수']=daejeon['10분내지하철수'].fillna(1)
test['10분내지하철수']=daejeon_test['10분내지하철수'].fillna(0)

In [1342]:
train['10분내지하철수']=train['10분내지하철수'].fillna(0)
test['10분내지하철수']=test['10분내지하철수'].fillna(0)

In [1343]:
print(train['10분내지하철수'].isnull().sum())
print(test['10분내지하철수'].isnull().sum())

0
0


In [1344]:
#강사님꺼
train.loc[train['10분내버스정류장수'].isnull(),"10분내버스정류장수"]=5
train.loc[train['10분내버스정류장수'].isnull(),:]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,주차면수 대비 등록차량 비율,주차면수 대비 총세대수,거주율


### 1000세대 이상 대단지로 분류. 대단지는 교통시설이 좋아 주차수요가 낮을것이다

In [1345]:
train.loc[train['총세대수']>=1000,'대단지']=0
train.loc[train['총세대수']<1000,'대단지']=1
test.loc[test['총세대수']>=1000,'대단지']=0
test.loc[test['총세대수']<1000,'대단지']=1

In [1346]:
danji=train.loc[train['대단지']==0].sort_values(by=['단지코드'])
print(danji['단지코드'].unique())
print(danji['단지코드'].count())

['C1005' 'C1013' 'C1017' 'C1109' 'C1129' 'C1156' 'C1176' 'C1177' 'C1210'
 'C1236' 'C1244' 'C1268' 'C1310' 'C1320' 'C1326' 'C1329' 'C1350' 'C1363'
 'C1386' 'C1406' 'C1407' 'C1430' 'C1439' 'C1448' 'C1451' 'C1565' 'C1579'
 'C1584' 'C1616' 'C1620' 'C1672' 'C1677' 'C1689' 'C1724' 'C1802' 'C1833'
 'C1859' 'C1863' 'C1875' 'C1889' 'C1933' 'C1939' 'C1969' 'C2000' 'C2034'
 'C2046' 'C2071' 'C2086' 'C2132' 'C2135' 'C2190' 'C2225' 'C2274' 'C2276'
 'C2289' 'C2310' 'C2316' 'C2328' 'C2329' 'C2352' 'C2358' 'C2361' 'C2362'
 'C2377' 'C2393' 'C2446' 'C2460' 'C2488' 'C2489' 'C2506' 'C2507' 'C2538'
 'C2565' 'C2579' 'C2583' 'C2597' 'C2612' 'C2618' 'C2620' 'C2621' 'C2666'
 'N1036' 'N2085' 'N2431']
956


In [1347]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,주차면수 대비 등록차량 비율,주차면수 대비 총세대수,거주율,대단지
0,C2515,545,아파트,경상남도,국민임대,30.0,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
1,C2515,545,아파트,경상남도,국민임대,35.0,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
2,C2515,545,아파트,경상남도,국민임대,35.0,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
3,C2515,545,아파트,경상남도,국민임대,45.0,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
4,C2515,545,아파트,경상남도,국민임대,45.0,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0


### mapping: 자격유형, 공급유형, 지역, 임대건물구분

In [1348]:
#강사님꺼
test.loc[196,'자격유형']='A'
test.loc[258,'자격유형']='C'

In [1349]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5,
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }
train['자격유형']=train['자격유형'].map(mapping).astype(int)
test['자격유형']=test['자격유형'].map(mapping).astype(int)

In [1350]:
mapping = { '국민임대':1, '임대상가':2, '행복주택':3, '공공임대(10년)':4, 
           '영구임대':5,'공공임대(50년)':6,
           '공공임대(분납)':7, '장기전세':8, '공공분양':9, '공공임대(5년)':10 }
train['공급유형'] =train['공급유형'].map(mapping).astype(int)
test['공급유형'] =test['공급유형'].map(mapping).astype(int)

In [1351]:
mapping1 = { '경기도':1, '경상남도':2, '부산광역시':3, '대전광역시':4, 
           '강원도':5,'충청북도':6,'충청남도':7, '전라남도':8, '광주광역시':9, 
            '경상북도':10, '전라북도':11,'대구광역시':12,'제주특별자치도':13,'서울특별시':14,'세종특별자치시':15,'울산광역시':16 }
train['지역'] =train['지역'].map(mapping1).astype(int)
test['지역'] =test['지역'].map(mapping1).astype(int)

In [1352]:
mapping2 = { '아파트':1, '상가':2}
train['임대건물구분'] =train['임대건물구분'].map(mapping2).astype(int)
test['임대건물구분'] =test['임대건물구분'].map(mapping2).astype(int)

In [1353]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수',
       '주차면수 대비 등록차량 비율', '주차면수 대비 총세대수', '거주율', '대단지'],
      dtype='object')

### 머신러닝

In [1354]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

In [1355]:
from sklearn.model_selection import cross_val_score

In [1356]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
import numpy as np
import time

In [1357]:
### 한글 폰트 설정
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
matplotlib.rcParams['axes.unicode_minus'] = False

In [1358]:
train.shape

(2896, 19)

In [1359]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,주차면수 대비 등록차량 비율,주차면수 대비 총세대수,거주율,대단지
0,C2515,545,1,2,1,30.0,276,17.0,1,9216000,82940,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
1,C2515,545,1,2,1,35.0,60,17.0,1,12672000,107130,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
2,C2515,545,1,2,1,35.0,20,17.0,1,12672000,107130,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
3,C2515,545,1,2,1,45.0,38,17.0,1,18433000,149760,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
4,C2515,545,1,2,1,45.0,19,17.0,1,18433000,149760,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0


In [1360]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수',
       '주차면수 대비 등록차량 비율', '주차면수 대비 총세대수', '거주율', '대단지'],
      dtype='object')

In [1361]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   단지코드             2896 non-null   object 
 1   총세대수             2896 non-null   int64  
 2   임대건물구분           2896 non-null   int32  
 3   지역               2896 non-null   int32  
 4   공급유형             2896 non-null   int32  
 5   전용면적             2896 non-null   float64
 6   전용면적별세대수         2896 non-null   int64  
 7   공가수              2896 non-null   float64
 8   자격유형             2896 non-null   int32  
 9   임대보증금            2327 non-null   object 
 10  임대료              2327 non-null   object 
 11  10분내지하철수         2896 non-null   float64
 12  10분내버스정류장수       2896 non-null   float64
 13  단지내주차면수          2896 non-null   float64
 14  등록차량수            2896 non-null   float64
 15  주차면수 대비 등록차량 비율  2896 non-null   float64
 16  주차면수 대비 총세대수     2896 non-null   float64
 17  거주율           

In [1362]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수,주차면수 대비 등록차량 비율,주차면수 대비 총세대수,거주율,대단지
0,C2515,545,1,2,1,30.0,276,17.0,1,9216000,82940,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
1,C2515,545,1,2,1,35.0,60,17.0,1,12672000,107130,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
2,C2515,545,1,2,1,35.0,20,17.0,1,12672000,107130,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
3,C2515,545,1,2,1,45.0,38,17.0,1,18433000,149760,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0
4,C2515,545,1,2,1,45.0,19,17.0,1,18433000,149760,0.0,3.0,624.0,205.0,0.328526,1.144954,0.968807,1.0


In [1363]:
# sel=['임대건물구분', '지역', '공급유형','자격유형', '10분내지하철수', '10분내버스정류장수',
#        '주차면수 대비 등록차량 비율', '주차면수 대비 총세대수', '거주율', '대단지']
#제거: 대단지, 총세대수, 임대건물구분, 
sel=['전용면적',
     '임대건물구분','지역','공급유형','자격유형',
     '주차면수 대비 등록차량 비율', '주차면수 대비 총세대수', '거주율']
X=train[sel]
y=train['등록차량수']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=0)

### 머신러닝ck

In [1364]:
import os, warnings
import re
warnings.filterwarnings(action='ignore')

In [1365]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [1366]:
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9988302078814335
테스트(score) : 0.9949411609655153


In [1367]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [1368]:
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.8715813845106714
테스트(score) : 0.8769996143257804


In [1369]:
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [1370]:
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9913638578162131
테스트(score) : 0.9865108869211511


In [1371]:
model = xgb.XGBRegressor()  # n_estimators=100
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [1372]:
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9998708422104113
테스트(score) : 0.99714967482368


### 교차검증

In [1373]:
now_time=time.time()
model_RF = RandomForestRegressor(random_state=30)
model_RF.fit(X_train, y_train)
score = cross_val_score(model_RF, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error") # neg_mean_squared_error
m_score = np.abs(score.mean())
print("RandomForestRegressor Score : {}".format(m_score))  # 점수
pro_time=time.time()-now_time
print(pro_time)

RandomForestRegressor Score : 25.15392622490598
3.864475727081299


In [1374]:
now_time=time.time()
model_RF = RandomForestRegressor(n_estimators=1000,random_state=30,n_jobs=-1)
model_RF.fit(X_train, y_train)
score = cross_val_score(model_RF, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error") # neg_mean_squared_error
m_score = np.abs(score.mean())
print("RandomForestRegressor Score : {}".format(m_score))  # 점수
pro_time=time.time()-now_time
print(pro_time)

RandomForestRegressor Score : 24.70087626781079
10.19309687614441


In [1375]:
now_time=time.time()
model_GB = GradientBoostingRegressor(max_depth=3,random_state=0)
model_GB.fit(X_train, y_train)
score = cross_val_score(model_GB, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error") # neg_mean_squared_error
m_score = np.abs(score.mean())
print("GradientBoostingRegressor Score : {}".format(m_score))  # 점수
pro_time=time.time()-now_time
print(pro_time)

GradientBoostingRegressor Score : 120.4130917271162
1.430715560913086


In [1376]:
m_lgbm1 = lgb.LGBMRegressor()
m_lgbm1.fit(X_train, y_train)
score = cross_val_score(m_lgbm1, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error")
m_score = np.abs(score.mean())  # 절대값
pro_time = time.time() - now_time
print("LightGBM Score : {}".format(m_score))  # 점수
print(pro_time)  # 걸린 시간

LightGBM Score : 44.973844135204914
1.8230085372924805


In [1377]:
xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train, y_train)
score = cross_val_score(xg_reg, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")
m_score = np.abs(score.mean())  # 절대값
pro_time = time.time() - now_time
print("XGBRegressor Score : {}".format(m_score))  # 점수
print(pro_time)  # 걸린 시간

XGBRegressor Score : 12.86705958936785
2.442284345626831


### 모델만들기

In [1378]:
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred[0:10]

array([2315.6384  ,  402.42416 ,  745.7189  ,  592.6351  ,  111.218475,
        699.3309  ,  570.64685 ,  315.03888 ,  334.85052 ,  508.43198 ],
      dtype=float32)

In [1379]:
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9998708422104113
테스트(score) : 0.99714967482368


In [1380]:
#mae_val=sum(abs(y_test-pred))/len(pred)
mae_val=np.mean(abs(y_test-pred))
mae_val

5.728410562757629

In [1381]:
#mse_val=sum((y_test-pred)**2)/len(pred)
mse_val=np.sum((y_test-pred)**2)/len(pred)
mse_val

570.0491957515914

In [1382]:
#rmse_val=mse_val ** 0.5
rmse_val=np.sqrt(mse_val)
rmse_val

23.875703042038182

In [1383]:
importance_df=pd.DataFrame(model.feature_importances_,index=X[sel].columns)
importance_df.sort_values(by=0,ascending=False)

,0
주차면수 대비 총세대수,0.332565
공급유형,0.264991
거주율,0.140987
자격유형,0.104665
주차면수 대비 등록차량 비율,0.096034
지역,0.058109
전용면적,0.002640
임대건물구분,0.000010


### 제출파일 만들기

In [1384]:
print(X_test.shape)
print(test.shape)

(724, 8)
(1008, 18)


In [1385]:
test.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수',
       '주차면수 대비 등록차량 비율', '주차면수 대비 총세대수', '거주율', '대단지'],
      dtype='object')

In [1386]:
sel=['전용면적',
     '임대건물구분','지역','공급유형','자격유형',
     '주차면수 대비 등록차량 비율', '주차면수 대비 총세대수', '거주율']
X_test=test[sel]
pred = model.predict(X_test)

In [1387]:
test['등록차량수'] = pred
test['단지별차량수평균'] = test.groupby("단지코드")['등록차량수'].transform(np.mean)
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()
test_new.head()

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,...,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,주차면수 대비 등록차량 비율,주차면수 대비 총세대수,거주율,대단지,등록차량수,단지별차량수평균
0,0,C1072,754,1,1,1,35.0,116,14.0,8,...,189840,0.0,2.0,683.0,0.300146,0.905836,0.981432,1.0,487.106140,573.124634
1,8,C1128,1354,1,1,1,35.0,368,9.0,8,...,189840,0.0,3.0,1216.0,0.875000,0.898080,0.993353,0.0,709.531311,681.748718
2,17,C1456,619,1,3,1,30.0,82,18.0,1,...,156200,0.0,16.0,547.0,1.945155,0.883683,0.970921,1.0,594.560486,491.930054
3,26,C1840,593,1,11,1,35.0,253,7.0,1,...,108130,0.0,3.0,543.0,1.018416,0.915683,0.988196,1.0,596.872009,548.886536
4,30,C1332,1297,1,1,1,35.0,282,11.0,8,...,203050,0.0,2.0,1112.0,0.373201,0.857363,0.991519,0.0,375.824860,1077.356567


### 제출용 오류 코드를 0으로 만들고 150행만들기

In [1388]:
add_dat={'code':['C2675','C2335','C1327'],'num':['0','0','0']}
add_df=pd.DataFrame(add_dat)
add_df

,code,num
0,C2675,0
1,C2335,0
2,C1327,0


In [1389]:
sub_df = test_new[ ['단지코드', '단지별차량수평균']]
sub_df.columns = ['code', 'num']
sub_df = pd.concat([sub_df, add_df]).reset_index()
sub_df = sub_df.drop(['index'], axis=1)
sub_df

,code,num
0,C1072,573.124634
1,C1128,681.748718
2,C1456,491.930054
3,C1840,548.886536
4,C1332,1077.356567
...,...,...
145,C1267,249.562958
146,C2189,162.305008
147,C2675,0
148,C2335,0


In [1391]:
sub_df.to_csv('0720_XGBR.csv', index=False)
sub_df.head()

,code,num
0,C1072,573.124634
1,C1128,681.748718
2,C1456,491.930054
3,C1840,548.886536
4,C1332,1077.356567
